In [ ]:
!pip install numpy networkx scikit-learn

In [ ]:
# schedule_demo.py
import networkx as nx
import numpy as np
from sklearn.cluster import SpectralClustering

# 1. 准备示例事件，每个事件代表一次“课-班-师”授课（一个学时）
events = [
    # event_id, 班级, 授课教师, 课程, 教室类型
    (0, 'A', 'T1', 'Math',      'normal'),
    (1, 'A', 'T2', 'English',   'normal'),
    (2, 'B', 'T1', 'Math',      'normal'),
    (3, 'B', 'T3', 'Physics',   'lab'),
    (4, 'C', 'T2', 'English',   'normal'),
    (5, 'C', 'T3', 'Physics',   'lab'),
    (6, 'A', 'T1', 'Chemistry', 'lab'),
    (7, 'B', 'T2', 'Chemistry', 'lab'),
    (8, 'C', 'T1', 'Math',      'normal'),
]
n_events = len(events)

# 2. 构建冲突图
G = nx.Graph()
for idx, cls, tea, course, room in events:
    G.add_node(idx, cls=cls, tea=tea, course=course, room=room)

# 对每对事件，若存在资源冲突则加边并赋权
from itertools import combinations
for i, j in combinations(range(n_events), 2):
    w = 0
    ei, ej = events[i], events[j]
    # 同班级：绝对冲突
    if ei[1] == ej[1]:
        w += 10
    # 同教师：绝对冲突
    if ei[2] == ej[2]:
        w += 8
    # 教室类型不一致：次级冲突（因为特定教室有限）
    if ei[4] != ej[4]:
        w += 2
    if w > 0:
        G.add_edge(i, j, weight=w)

# 3. 构建“相似度矩阵”供谱聚类使用
# 我们把权重 w 转成相似度 sim = 1/(1+w)
A = nx.to_numpy_array(G, weight='weight')
sim = 1.0 / (1.0 + A)  # 对角会是 1/(1+0)=1

# 4. 谱聚类：假设我们有 3 天 × 3 节 = 9 个时间槽
n_slots = 9
sc = SpectralClustering(
    n_clusters=n_slots,
    affinity='precomputed',
    assign_labels='kmeans',
    random_state=42
)
labels = sc.fit_predict(sim)
# labels[i] = 0..8 其中之一，表示事件 i 分到哪个时间槽

# 5. 映射到可读时段
timeslots = [f'Day{d}_P{p}'
             for d in range(1,4)    # 3 天
             for p in range(1,4)]   # 每天 3 节
schedule = {ts: [] for ts in timeslots}
for idx, slot in enumerate(labels):
    schedule[timeslots[slot]].append(events[idx])

# 6. 输出初排结果
print("=== 初排课表 ===")
for ts in timeslots:
    print(f"{ts}:")
    for ev in schedule[ts]:
        print(f"  班级{ev[1]} 老师{ev[2]} 课目{ev[3]} 教室类型{ev[4]}")
    if not schedule[ts]:
        print("   （空）")

# 7. 简单检查：是否有硬冲突？
conflicts = 0
for ts in timeslots:
    lst = schedule[ts]
    # 检查同一时段内是否同班/同师重复
    classes = [ev[1] for ev in lst]
    teachers = [ev[2] for ev in lst]
    if len(classes) != len(set(classes)):
        conflicts += 1
    if len(teachers) != len(set(teachers)):
        conflicts += 1
print(f"\n硬冲突时段数（应为0）：{conflicts}")

=== 初排课表 ===
Day1_P1:
  班级A 老师T1 课目Chemistry 教室类型lab
Day1_P2:
  班级C 老师T1 课目Math 教室类型normal
Day1_P3:
  班级B 老师T3 课目Physics 教室类型lab
Day2_P1:
  班级A 老师T1 课目Math 教室类型normal
Day2_P2:
  班级B 老师T1 课目Math 教室类型normal
Day2_P3:
  班级A 老师T2 课目English 教室类型normal
Day3_P1:
  班级B 老师T2 课目Chemistry 教室类型lab
Day3_P2:
  班级C 老师T2 课目English 教室类型normal
Day3_P3:
  班级C 老师T3 课目Physics 教室类型lab

硬冲突时段数（应为0）：0


/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:372: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  _, diffusion_map = eigsh(


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

class DualScheduleCNN(nn.Module):
    def __init__(self, height=5, width=10, output_channels=64):
        super().__init__()
        # 每个课表的输入通道数：2（教师课表：[course_name_id, room_name_id]，课室课表：[course_name_id, teacher_name_id]）
        in_channels = 2

        # 教师课表CNN
        self.teacher_conv = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 输出：H/2 × W/2
            nn.Conv2d(32, output_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 输出：H/4 × W/4
        )

        # 课室课表CNN
        self.room_conv = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 输出：H/2 × W/2
            nn.Conv2d(32, output_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 输出：H/4 × W/4
        )

        # 输出维度
        self.output_height = height // 4
        self.output_width = width // 4
        self.output_channels = output_channels

    def forward(self, teacher_schedule, room_schedule, course_ids, teacher_ids, room_ids):
        """
        输入：
            teacher_schedule: (batch_size, H, W, 2) - 教师课表，[course_name_id, room_name_id]
            room_schedule: (batch_size, H, W, 2) - 课室课表，[course_name_id, teacher_name_id]
            course_ids, teacher_ids, room_ids: (batch_size,) - 待安排课程的ID
        输出：
            x: (batch_size, 2 × H/4 × W/4, output_channels) - 拼接的CNN特征
            course_ids, teacher_ids, room_ids: 原样返回
        """
        # 调整输入格式
        teacher_input = teacher_schedule.permute(0, 3, 1, 2)  # (batch_size, 2, H, W)
        room_input = room_schedule.permute(0, 3, 1, 2)  # (batch_size, 2, H, W)

        # 教师课表 CNN
        teacher_output = self.teacher_conv(teacher_input)  # (batch_size, output_channels, H/4, W/4)
        teacher_output = teacher_output.permute(0, 2, 3, 1)  # (batch_size, H/4, W/4, output_channels)
        teacher_output = teacher_output.reshape(teacher_output.size(0), -1, self.output_channels)  # (batch_size, H/4 × W/4, output_channels)

        # 课室课表 CNN
        room_output = self.room_conv(room_input)  # (batch_size, output_channels, H/4, W/4)
        room_output = room_output.permute(0, 2, 3, 1)  # (batch_size, H/4, W/4, output_channels)
        room_output = room_output.reshape(room_output.size(0), -1, self.output_channels)  # (batch_size, H/4 × W/4, output_channels)

        # 拼接特征
        x = torch.cat([teacher_output, room_output], dim=1)  # (batch_size, 2 × H/4 × W/4, output_channels)

        return x, course_ids, teacher_ids, room_ids

def prepare_schedule_input(teacher_schedules, room_schedules, course_ids, teacher_ids, room_ids, height=5, width=10):
    """
    将Schedule对象转换为CNN输入张量
    输入：
        teacher_schedules, room_schedules: List[Schedule] - 批量课表对象
        course_ids, teacher_ids, room_ids: List[int] - 待安排课程的ID
    输出：
        teacher_tensor: (batch_size, H, W, 2) - 教师课表张量，[course_name_id, room_name_id]
        room_tensor: (batch_size, H, W, 2) - 课室课表张量，[course_name_id, teacher_name_id]
        course_tensor, teacher_tensor_ids, room_tensor_ids: (batch_size,) - ID张量
    """
    batch_size = len(teacher_schedules)
    teacher_tensor = torch.zeros(batch_size, height, width, 2, dtype=torch.float32)
    room_tensor = torch.zeros(batch_size, height, width, 2, dtype=torch.float32)

    for b in range(batch_size):
        teacher_table = teacher_schedules[b].get_table()
        room_table = room_schedules[b].get_table()

        for i in range(height):
            for j in range(width):
                # 教师课表：[course_name_id, room_name_id]
                course = teacher_table.iloc[i, j]
                if course is not None:
                    teacher_tensor[b, i, j] = torch.tensor([
                        course.name_id,  # course_name_id
                        course.room_id   # room_name_id
                    ], dtype=torch.float32)

                # 课室课表：[course_name_id, teacher_name_id]
                course = room_table.iloc[i, j]
                if course is not None:
                    room_tensor[b, i, j] = torch.tensor([
                        course.name_id,   # course_name_id
                        course.teacher_id # teacher_name_id
                    ], dtype=torch.float32)

    course_tensor = torch.tensor(course_ids, dtype=torch.long)
    teacher_tensor_ids = torch.tensor(teacher_ids, dtype=torch.long)
    room_tensor_ids = torch.tensor(room_ids, dtype=torch.long)

    return teacher_tensor, room_tensor, course_tensor, teacher_tensor_ids, room_tensor_ids

# 示例课程类
class Course:
    def __init__(self, name_id: int, teacher_id: int, room_id: int):
        self.name_id = name_id
        self.teacher_id = teacher_id
        self.room_id = room_id
        self.schedule_row = None
        self.schedule_col = None

    def update(self):
        pass

# 示例使用
if __name__ == "__main__":
    from schedule import Schedule  # 假设Schedule类在schedule.py中

    # 模拟数据
    H, W = 5, 10
    batch_size = 2

    # 创建课表
    teacher_schedules = [Schedule(row=H, col=W, name=f"Teacher_{i}") for i in range(batch_size)]
    room_schedules = [Schedule(row=H, col=W, name=f"Room_{i}") for i in range(batch_size)]

    # 填充示例课程
    for b in range(batch_size):
        teacher_schedules[b].set_course(Course(1, 2, 3), 0, 0)
        room_schedules[b].set_course(Course(1, 2, 3), 0, 0)

    # 待安排课程的ID
    course_ids = [4, 5]
    teacher_ids = [6, 7]
    room_ids = [8, 9]

    # 准备输入
    teacher_tensor, room_tensor, course_tensor, teacher_tensor_ids, room_tensor_ids = prepare_schedule_input(
        teacher_schedules, room_schedules, course_ids, teacher_ids, room_ids, H, W
    )

    # 初始化CNN
    model = DualScheduleCNN(height=H, width=W, output_channels=64)

    # 前向传播
    output, c_ids, t_ids, r_ids = model(teacher_tensor, room_tensor, course_tensor, teacher_tensor_ids, room_tensor_ids)
    print(f"CNN output shape: {output.shape}")  # (batch_size, 2 × H/4 × W/4, output_channels)
    print(f"Course IDs shape: {c_ids.shape}")  # (batch_size,)

# 新增區段